In [1]:
!pip install rich

In [3]:
import re
import json 
import asyncio 
from dataclasses import dataclass
from typing import Dict, List

from autogen_core import SingleThreadedAgentRuntime
from autogen_core import MessageContext, AgentId
from autogen_core import DefaultTopicId, RoutedAgent, TypeSubscription, default_subscription, message_handler
from autogen_core.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,    
)
from autogen_ext.models.openai import OpenAIChatCompletionClient

from dotenv import load_dotenv
load_dotenv()

True

## Message Protocols

Agent 동작 간에 주고 받는 메시지 데이터들 정의

In [4]:
@dataclass 
class Topic:
    content: str 

@dataclass 
class Participants:
    content: str  # 필요한가?
    participants_personas: List[Dict]

@dataclass 
class Persona:
    name: str 
    panel_type: str 
    description: str 
    topic: str 
    
@dataclass 
class PanelResponse:
    persona: Persona
    answer: str 

## Agents

### Persona Agent

In [10]:
# 주어진 텍스트를 기반으로 토론 주제와 페르소나 정보를 추출
# 정보는 Participant 형식으로 반환 
@default_subscription
class PersonaAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient,) -> None:
        super().__init__("Persona Agent")
        self._model_client = model_client
        self._participants: List[Participants] = [] 
        self._system_messages = [
            SystemMessage(
                content=(
                    f"다음 텍스트를 기반으로 가능한 1개의 토론 주제를 설정하세요."
                    "토론 주제에 적합한 주요 의견을 생성하고, 각 의견을 대변할 가상의 토론 participant 를 생성하세요. "
                    "출력은 {{\"debate_title\": \"title\", \"participants\": [{\"participant_name\": \"parti_A\", \"description\": \"...\"},...]}} 으로 출력하세요."
                )
            )
        ]
    
    @message_handler
    async def handle_topic_request(self, message: Topic, ctx: MessageContext) -> None:
        print(f"{'-'*80}\nPersona Agent {self.id} received topic:\n{message.content}")
        history_msg = UserMessage(content=message.content, source="user")
        # Make an inference using the model.
        model_result = await self._model_client.create(self._system_messages + [history_msg])
        assert isinstance(model_result.content, str)
        try:
            participants_json = json.loads(model_result.content.strip())
        except Exception as e:
            raise ValueError("Participants data is invalid\n")
        
        print(f"{'-'*80}\nPersonaAgent {self.id} publishes initial participants.\n")
        print(f"{participants_json}")
        await self.publish_message(Participants(content=message.content, participants_personas=participants_json), topic_id=DefaultTopicId())

### Panel Agent

In [11]:
# Persona Agent 가 정의한 특정 페르소나에 입각하여 발언하는 Agent
@default_subscription
class PanelAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None: 
        super().__init__("Panel Agent")
        self.model_client = model_client        
    
    @message_handler
    async def handle_persona_define_request(self, message: Persona, ctx: MessageContext) -> PanelResponse: 
        # 주어진 persona description 으로 SystemMessage 를 설정하고 topic에 대한 answer 를 생성 
        if message.panel_type == 'Panel':
            system_prompt = f"""You are a friendly AI. 
Your persona description is: {message.description}            
Represent the general characteristics of your persona.
State your position on the given topic. 
Please present a one-sided argument rather than a neutral argument.
State your opinion in as much detail and persuasive as possible. Speak in Korean.""" 
            user_prompt = f"Debate Topic: {message.topic}\n" 
            model_result = await self.model_client.create(
                [SystemMessage(content=system_prompt), UserMessage(content=user_prompt, source='user')]
            )
            assert isinstance(model_result.content, str)
            print(f"{'-'*80}\nPanel name: {message.name} , description: {message.description}\n")
            print(f"{model_result.content}")
            return PanelResponse(answer=model_result.content, persona=message)
            #await self.publish_message(PanelResponse(answer=model_result.content, persona=message), topic_id=DefaultTopicId())
        elif message.panel_type == 'Critique':
            pass
        pass 

### Debate Moderator Agent

In [12]:
# 토론 Moderator Agent
# Persona Agent 가 정의한 개별 페르소나 별로 Agent 를 생성시킨다. 

from uuid import uuid4
# handle msg(participants) then Generate multiple panel based on Personas
@default_subscription
class DebateModeratorAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("Debate Moderator Agent")
        
    @message_handler
    async def handle_participants_request(self, message: Participants, ctx: MessageContext) -> None:
        print(f"{'-'*80}\nDebate Moderator Agent {self.id} received participants:")
        print(f"message.participants_personas \n{message.participants_personas['debate_title']}")        
        
        for part in message.participants_personas['participants']:
            agent_id = AgentId('Panel', str(uuid4()))
            print(f"{'-'*80}\nGenerate Panel Agents")
            print(f"participant name: {part['participant_name']}\n")
            print(f"participant description: {part['description']}\n")       
            print(f"AgentId: {agent_id}\n")     
            persona = Persona(name=part['participant_name'], description=part['description'], topic=message.content, panel_type='Panel')
            # Genrate Panel Agents based on the participants 
            #results = await asyncio.gather(*[self.send_message(message=persona, recipient=agent_id)])
            await self.publish_message(persona, topic_id=DefaultTopicId())
        #
        
        #



### Setting up Runtime

In [13]:
# MaD 런타임을 생성하고, 페르소나, 패널, 모더레이터 Agent 를 각각 등록한다. 

runtime = SingleThreadedAgentRuntime()

await PersonaAgent.register(
    runtime, 
    "PersonaAgent", 
    lambda: PersonaAgent(
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    )
)

await PanelAgent.register(
    runtime, 
    "PanelAgent",
    lambda: PanelAgent(
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini")
    )
)

await DebateModeratorAgent.register(
    runtime, 
    "DebateModeratorAgent",
    lambda: DebateModeratorAgent(
    )
)

AgentType(type='DebateModeratorAgent')

## Run

In [ ]:
# 토론 Topic 을 선언하고, @default_subscription 으로 등록된 Agent 들이 수신할 수 있도록 메시지를 publish 한다.
topic = """계엄 예측한 김민석 “가장 큰 동기 ‘김건희 감옥 가기 싫어서’”
김민석 더불어민주당 최고위원은 4일 “윤석열 대통령이 ‘반국가 세력’이라는 용어를 쓰기 시작한 것이 계엄론의 논리적인 밑밥을 까는 것이고 빌드업이었던 것”이라고 주장했다.

김 최고위원은 이날 오전 문화방송(MBC) 라디오 ‘김종배의 시선집중’에 나와 “(윤 대통령의 비상계엄 선포는) 비정상적인 권력 집착, 
그리고 사실 시작은 김건희씨의 비정상적 권력 집착에서 시작된 것”이라며 이렇게 말했다.

그는 지난 8월21일 민주당 최고위원회의에서 “차지철 스타일의 야당 입틀막 국방장관으로의 갑작스러운 교체와 대통령의 뜬금없는 반국가 세력 발언으로 이어지는 
최근 정권 흐름의 핵심은 국지전과 북풍 조성을 염두에 둔 계엄령 준비 작전이라는 것이 근거 있는 확신”이라며, 민주당 안에서 ‘계엄 준비설’을 주도해왔다.

김 최고위원은 이날 인터뷰에서 이와 관련해 “(윤 정권이) 워낙 국정을 못하기 때문에 계엄과 테러, 
‘사법적으로 상대편 죽이기’ 외에는 정권 교체를 막을 방법이 없다고 생각하고 있다는 등등의 종합적 판단을 했던 것이 제가 문제를 제기했던 배경”이라고 말했다.

그는 또 당시 계엄 준비설을 주장한 배경과 관련해 “거기(계엄)에 동원될 세력으로서의 ‘충암파’들을 재배치하는 것이 이상하다고 판단했다”며 
“가장 큰 핵심적 동기는 ‘김건희 감옥 가기 싫다’다”라고 설명했다.

아울러 “채 상병 문제와 관련돼 있는데 아마 대통령을 포함해 국방부 장관 등등이 다 연루돼 있을 거라고 저희는 본다”며 
“결국은 진실이 규명되면 감옥에 갈 수밖에 없는 자들이 자기 보존을 위해 사고를 친 것”이라고도 했다.

비상 계엄 선포를 건의한 것으로 알려진 김용현 국방장관은 윤 대통령의 충암고 1년 선배고, 정보기관인 국군방첩사령관에 임명된 여인형 중장도 충암고 출신이다. 
방첩사는 박근혜 정부 시절 계엄령 검토 문건을 작성한 국군기무사령부(기무사)의 후신으로, 계엄이 선포되면 주요 사건 수사를 지휘하고 정보·수사기관을 조정·통제할 
합동수사본부도 방첩사에 꾸려진다. 또 대북 특수정보 수집의 핵심 기관인 777사령부 수장인 박종선 사령관, 
현행 계엄법상 국방부 장관과 함께 대통령에게 계엄 발령을 건의할 수 있는 이상민 행정안전부 장관도 충암고 출신이다.


비상 계엄 선포에 대한 추가적인 방지책이 있냐는 질문에는 “이번에 대통령이 2~3시간 동안 (계엄 선포를) 했다가 무산돼버린 1차 시도라고 본다. 
아직 잔불이 끝나지 않았다”며 “지금 21세기 대명천지에 제가 이 문제를 처음 제기했을 때만 해도 저를 오히려 이상하게 보는 분들이 대부분이었는데 
우리가 이런 것을 상정해서 더 나은 보완책을 고민해야 하는가 그게 너무나 참 황당한 상황인 것”이라고 말했다."""
runtime.start()
await runtime.publish_message(Topic(content=topic), DefaultTopicId())
await runtime.stop_when_idle()

--------------------------------------------------------------------------------
Persona Agent PersonaAgent/default received topic:
계엄 예측한 김민석 “가장 큰 동기 ‘김건희 감옥 가기 싫어서’”
김민석 더불어민주당 최고위원은 4일 “윤석열 대통령이 ‘반국가 세력’이라는 용어를 쓰기 시작한 것이 계엄론의 논리적인 밑밥을 까는 것이고 빌드업이었던 것”이라고 주장했다.

김 최고위원은 이날 오전 문화방송(MBC) 라디오 ‘김종배의 시선집중’에 나와 “(윤 대통령의 비상계엄 선포는) 비정상적인 권력 집착, 
그리고 사실 시작은 김건희씨의 비정상적 권력 집착에서 시작된 것”이라며 이렇게 말했다.

그는 지난 8월21일 민주당 최고위원회의에서 “차지철 스타일의 야당 입틀막 국방장관으로의 갑작스러운 교체와 대통령의 뜬금없는 반국가 세력 발언으로 이어지는 
최근 정권 흐름의 핵심은 국지전과 북풍 조성을 염두에 둔 계엄령 준비 작전이라는 것이 근거 있는 확신”이라며, 민주당 안에서 ‘계엄 준비설’을 주도해왔다.

김 최고위원은 이날 인터뷰에서 이와 관련해 “(윤 정권이) 워낙 국정을 못하기 때문에 계엄과 테러, 
‘사법적으로 상대편 죽이기’ 외에는 정권 교체를 막을 방법이 없다고 생각하고 있다는 등등의 종합적 판단을 했던 것이 제가 문제를 제기했던 배경”이라고 말했다.

그는 또 당시 계엄 준비설을 주장한 배경과 관련해 “거기(계엄)에 동원될 세력으로서의 ‘충암파’들을 재배치하는 것이 이상하다고 판단했다”며 
“가장 큰 핵심적 동기는 ‘김건희 감옥 가기 싫다’다”라고 설명했다.

아울러 “채 상병 문제와 관련돼 있는데 아마 대통령을 포함해 국방부 장관 등등이 다 연루돼 있을 거라고 저희는 본다”며 
“결국은 진실이 규명되면 감옥에 갈 수밖에 없는 자들이 자기 보존을 위해 사고를 친 것”이라고도 했다.

비상 계엄 선포를 건의한 것으로 알려진 김용현 국방장관은 윤 대통령의